In [ ]:
import os, math, glob, re
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt
import pydicom

from kaggle_datasets import KaggleDatasets

from sklearn.model_selection import train_test_split


from random import shuffle
import tensorflow as tf

In [ ]:
IMAGE_SIZE  = 256
IMAGE_DEPTH = 64

mri_types = ['FLAIR','T1w','T1wCE','T2w']
CHANNELS  = len(mri_types)

local_directory = "../input/rsna-miccai-brain-tumor-radiogenomic-classification"
local_label_path = local_directory+"/train_labels.csv"
local_submission_path = local_directory+"/sample_submission.csv"

# Load DICOM

In [ ]:
# load 1 dicom img, e.g. image-1.dcm
def load_dicom_slice(path, img_size=256):

    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = cv2.resize(data, (img_size, img_size))
    
    return data

# load all dicoms in a modality folder, e.g. FLAIR/*.dcm
def load_dicom_modality(mri_type, scan_id, img_depth, img_size, split):

    files = sorted(tf.io.gfile.glob(f"{local_directory}/{split}/{scan_id}/{mri_type}/*.dcm"), 
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])


    
    num_files = len(files)
    num_files_middle = num_files//2
    img_depth_middle = img_depth//2
    
    start_depth = max(0, num_files_middle - img_depth_middle)
    end_depth   = min(num_files, num_files_middle + img_depth_middle)
    img3d = np.stack([load_dicom_slice(dicom_img, img_size=img_size) 
                      for dicom_img in files[start_depth:end_depth]]).T
    
    if img3d.shape[-1] < img_depth:
        n_zero = np.zeros((img_size, img_size, img_depth - img3d.shape[-1]))
        img3d  = np.concatenate((img3d, n_zero), axis=-1)        

    return img3d

# load all modality for a single sample, e.g. 00010/*/*.dcm
def load_dicom_3D(scan_id, img_depth=128, img_size=256, split="test"):
    print(scan_id, end=" ")
    dicom_channels = [load_dicom_modality(scan_id=scan_id,
                                         img_depth=img_depth, 
                                         img_size=img_size, 
                                         split=split,
                                         mri_type=mtype) 
                      for mtype in mri_types]
        
    img = np.stack(dicom_channels, axis=-1)
    # Normalize
    if np.min(img) < np.max(img):
        img = img - np.min(img)
        img = img / np.max(img)
    return img

# Convert to TFRecords

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(image):
    feature = {
        'image': _bytes_feature(image.tobytes()),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
def sorted_alphanumeric(data):  #
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

#sorte the list 

In [ ]:
! mkdir -p ./tfrecords/test/
outpath_train = "./tfrecords/test"
with tf.io.TFRecordWriter(str(outpath_train + os.sep + 'brain_test.tfrec'),
                          options=tf.io.TFRecordOptions(compression_type="GZIP")) as writer:
    Path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"
    train_dir = os.listdir(Path)
    train_dir = sorted_alphanumeric(train_dir)
    print(f"traindir : {train_dir}")
    
    for x in train_dir:
        img = load_dicom_3D(x, img_size=IMAGE_SIZE, img_depth=IMAGE_DEPTH, split="test")
        example = serialize_example(img)
        writer.write(example)

        


In [ ]:
AUTO = tf.data.AUTOTUNE
def deserialize_example(serialized_string):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
    }
    parsed_record = tf.io.parse_single_example(serialized_string, image_feature_description)
    image = tf.io.decode_raw(parsed_record['image'], tf.float64)

    image = tf.reshape(image,[256, 256, 64, 4])


    return image

BATCH_SIZE = 1 
train_set = tf.data.TFRecordDataset("./tfrecords/test/brain_test.tfrec",compression_type="GZIP", num_parallel_reads=AUTO).map(deserialize_example).batch(BATCH_SIZE).prefetch(AUTO)


In [ ]:
model = tf.keras.models.load_model('../input/test-2/Test2.h5',  custom_objects={'leaky_relu': tf.nn.leaky_relu})
pred = model.predict(train_set)

In [ ]:
print(pred.shape)
print("dab")
sub = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
print(sub)
sub['MGMT_value'] = pred
sub.to_csv('submission.csv', index=False)
print(sub)